### Import all Packages ###

In [5]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import collections
import nest_asyncio
%reload_ext tensorboard
nest_asyncio.apply()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

### Load in TFF - EMNIST Data Set ###

In [6]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(only_digits=True)

### Create a Server Class ###

In [19]:
class server:
    
    def __init__(self, numOfClients):
        self.numOfClients = numOfClients #Num of Client
        self.clientIds = list(range(0,self.numOfClients)) #Client List
        self.updateRoundNum = 0  # Update Round Number
        self.serverModel = self.createBaseModel() #Create a Keras Model for MNIST
        self.clientActive = [] #If client is available or not based on time - day/night
        self.clientCIF = []
        self.clientSelected = []
        self.maxClientsPerRound = 10
        self.updateFromClients=[]
        self.__serverTestData_X=[]
        self.__serverTestData_Y=[]
        self.__serverTestData = self.setServerTestDataTFF(emnist_test)
        self.predictionAcc=[]
        self.predictionLoss=[]
        self.__serverTrainData = self.setServerTrainDataTFF(emnist_train)
        
    def createBaseModel(self):
        #return tf.keras.models.Sequential([
        #    Dense(64, activation='relu',input_shape=(784,)),
        #    Dense(64, activation='relu'),
        #    Dense(10, activation='softmax'),])
        return tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(784,)),
            tf.keras.layers.Dense(10, kernel_initializer='zeros'),
            tf.keras.layers.Softmax()])
    
    def setServerTestData(self,Xtest,Ytest):  # NOT USED IN THIS CASE
        self.__serverTestData_X=Xtest
        self.__serverTestData_Y=Ytest
        
    def setServerTestDataTFF(self,serverTestData):  #Creates a test set for server
        BATCH_SIZE = 2500
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(10).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        serverData1 = serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[10])
        for i in range (0,3382):
            serverData1 = serverData1.concatenate(serverTestData.create_tf_dataset_for_client(serverTestData.client_ids[i]))
            
        serverData =tfds.as_numpy(preprocess(serverData1))
        return serverData
     
    def setServerTrainDataTFF(self,serverTrainData):  
        #Creates a train set for server -- used for general only server training case i.e. no client all data trained on server
        # and for pre-training case evaluation i.e pre-trained model distributed to clients instead of random initializing
        #uses the emnist train dataset instead of the test data set
        
        BATCH_SIZE = 100
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(50).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        serverData2 = serverTrainData.create_tf_dataset_for_client(serverTrainData.client_ids[1])
        for i in range (2000,2500):
            serverData2 = serverData2.concatenate(serverTrainData.create_tf_dataset_for_client(serverTrainData.client_ids[i]))
            
        serverData =tfds.as_numpy(preprocess(serverData2))
        return serverData
    
    def initialBroadcast(self):
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name, Using Client_List Dictionary
            #clientName = "client_"+str(self.clientIds[i])  #Create Client Name
            #print(clientName)
            clientName.setInitialModel(self.serverModel) #setInitialModel-> Method of client Class
            #eval(clientName).setInitialModel(self.serverModel) #setInitialModel-> Method of client Class
            
    def getClientActiveStatus(self):  #if the client is available for update or not randomly set in client
        print("--------------------------------------------\n","Round NO:",self.updateRoundNum)
        self.clientActive = []
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name
            #print(clientName)
            if(clientName.sendActiveStatus() == 1): #getActiveStatus() -> method of client class
                self.clientActive.append(i)
        #print("Active Clients in Round :--: ", self.clientActive)
    
    
    def getClientCIF(self): # Get Client Importance Factor for active clients
        self.clientCIF=[]
        for i in self.clientActive:
            clientName = client_list["client_"+str(i)]
            c_cif = clientName.sendCIF()   #sendCIF() - method of client
            self.clientCIF.append(c_cif)
        #print("Clients with Acceptable CIF: ", self.clientCIF)
        
    def getClientSelected(self): # Select The Top (N = maxClientsPerRound) with the highest CIF value
        self.clientSelected = []
        # Use non-class, local variables to leave class variables clean
        clientCIF, clientActive = ( list(t) for t in zip(*sorted(zip(self.clientCIF, self.clientActive)))) # Sort Active Clients and their CIF values, by CIF
        self.clientSelected  = clientActive[-self.maxClientsPerRound:] # Select the N=maxClientsPerRound of clients with highest CIF
        self.clientCIF = clientCIF[-self.maxClientsPerRound:] # Select the N=maxClientsPerRound of highest CIF
    
    def getClientSelectedMaxFrac(self): # Select The Top (N = maxClientsPerRound) with the highest CIF value
        self.clientSelected = []
        # Use non-class, local variables to leave class variables clean
        clientCIF, clientActive = ( list(t) for t in zip(*sorted(zip(self.clientCIF, self.clientActive)))) # Sort Active Clients and their CIF values, by CIF
        maxCIF = clientCIF[-1:][0]
        limitCIF = maxCIF*0.9
        newClientCIF = [i for i in clientCIF if i>limitCIF]
        newSelectedClients = clientActive[-len(newClientCIF):]
        self.clientSelected = newSelectedClients
        self.clientCIF = newClientCIF
        print('RoundMaxCIF:',maxCIF,' ||',end=" ")
        print('RoundLimitCIF:',limitCIF,' ||',end=" ")
        print('numberOfClientSlected:',len(self.clientSelected))
        
    def getRandomClients(self):
        self.clientSelected = np.random.randint(0, 100, 10)
        self.clientCIF=[]*0
        self.clientActive=[]*0
    
    def getModelUpdateFromClients(self):
        
        #print("Active Clients: ",self.clientActive)
        #print("CIF of Active Clients: ", self.clientCIF)
        #print("Selected Clients: ", self.clientSelected)
        #print("Training Clients with CIF > 10")
        self.updateRoundNum +=1
        self.updateFromClients=[]
        #self.dataPointsClients=[]
        
        for i in range (0,len(self.clientSelected)):
            #if self.clientCIF[i] >= 10:  #Select Client only if CIF > some value 10 chosen for testing ONLY
            clientName = client_list["client_"+str(int(self.clientSelected[i]))]
            self.updateFromClients.append(clientName.sendClientUpdate()) #Get a list of updates from selected clients

        avgModelWeights = self.computeFedAvg(self.updateFromClients)
        #print("average model weights = \n",avgModelWeights)
        self.serverModel.set_weights(avgModelWeights)   
        
    def computeFedAvg(self,updates):  #Compute Federated Averaging from Available Clients
        totalDataPoints = 0
        scaleFactor=[]
        for i in range (0,len(updates)):
            totalDataPoints += updates[i][1]   #Sum the total Data Points on All Available Clients
            scaleFactor.append(updates[i][1])  #Store individual number datapoints for clients

        scaleFactor = np.array(scaleFactor)/totalDataPoints #Create the scale factor
       
        sumOfAvgWeights = []*len(updates[0][0]) 
        
        for j in range (0,len(updates[0][0])): #range of layers
            k=np.zeros_like(updates[0][0][j])
            #print("ShapeofK:",k.shape)
            for i in range(0,len(updates)): #range of clients
                k=k+updates[i][0][j]*scaleFactor[i]
                #print("ShapeofK:",k.shape)
            sumOfAvgWeights.append(k)
      
        return sumOfAvgWeights   
    
    def testServerModel(self):
        self.serverModel.compile(loss = 'sparse_categorical_crossentropy',
                                optimizer = tf.keras.optimizers.SGD (learning_rate=.01),
                                metrics=['accuracy'])
        testData = next(iter(self.__serverTestData))
        return self.serverModel.evaluate(testData['x'],
                                         testData['y'],)
        
        
    def updateAllClients(self):
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]  #Create Client Name
            clientName.setModelUpdateWeights(self.serverModel.get_weights())
    
    def getDataCost(self):
        totalDataCost=0
        for i in self.clientIds:
            clientName = client_list["client_"+str(self.clientIds[i])]
            totalDataCost += clientName.getDataCost()
        print("Total Data Cost = ",totalDataCost)
        return totalDataCost
    
    def performServerTraining(self):
        trainData = next(iter(self.__serverTrainData))
        self.serverModel.compile(loss = 'sparse_categorical_crossentropy',
                                 optimizer = tf.keras.optimizers.SGD (learning_rate=.01),
                                 metrics=['accuracy'])
        self.serverModel.fit(trainData['x'],
                                       trainData['y'],
                                       epochs=1,
                                       verbose=0,shuffle=True)


### Create a Client Class ###

In [20]:
class client:
    
    def __init__(self,ID):
        self.id = ID
        self.__clientModel = tf.keras.models.Sequential()
        self.__clientModelWeights=[]
        self.activeStatus = 0
        self.__clientDataX=[] #Private to Client
        self.__clientDataY=[]
        self.clientCIF=0
        self.meanSINR = int(np.random.normal(15, 2.5, 1)) #Select random int from normal dist 
        self.roundSINR = 0
        self.propFair = 0
        self.epochs = 10
        self.lr = 0.01
        self.__clientData = self.__clientDataFromTFF(emnist_train)
        self.__currentData = next(iter(self.__clientData))
        self.dataQuality = 0
        self.dataCost = 0
        self.dataCostFactor=0.2
        
        
    def __clientDataFromTFF(self,tffDataset):
        NUM_EPOCHS = self.epochs
        BATCH_SIZE = 20
        #BATCH_SIZE = 50
        SHUFFLE_BUFFER = 100
        PREFETCH_BUFFER= 10

        def preprocess(dataset):
            def batch_format_fn(element):
                """Flatten a batch `pixels` and return the features as an `OrderedDict`."""
                return collections.OrderedDict(
                    x=tf.reshape(element['pixels'], [-1, 784]),
                    y=tf.reshape(element['label'], [-1, 1]))

            return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
              BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)
        
        return tfds.as_numpy(preprocess(tffDataset.create_tf_dataset_for_client(tffDataset.client_ids[self.id])))
    
    
    #Methods Private to client
    def __setActiveStatus(self): 
        self.activeStatus = int(np.random.randint(0,2,1))  #01.01.21 Random Client Activation
        #self.activeStatus = 1
    
    def __setRoundSINR(self):
        self.roundSINR = np.round((np.random.normal(self.meanSINR, 1, 1)[0]), 2)
        #print("Round SNR for Client: ",self.id," is: ", self.roundSINR)
                               
    def __setPropFair(self):
        self.propFair = self.roundSINR / self.meanSINR
        
    def __setdataQuality(self):\
        
        ## Calculate for entropy - Start ##
        
#         for values in classification_probabilities:
#           for value in values:
#             score += value * np.log(value)
#         score = score * -1

        ## Calculate for entropy - End ##
        
        #self.dataQuality = 20 # Set 20 so that CIF value will always be passing
        
        # Next Client Data to be called  <<_ NOt written yet
        self.__currentData = next(iter(self.__clientData))
        predictions = self.__clientModel.predict(self.__currentData['x'])  # Get Class Predictions using Predict
        entropy = []*len(predictions)  # Entropy Will hold the entropy scores for all samples in the given round for this round
       
        for i in range (0,len(predictions)):
            sampleClassProb = predictions[i] #Take Class probability for every sample 
            score=0
            for value in sampleClassProb:  # Calculate the score - same as what jonny wrote in the pseudo code above
                #print('Value:',value)
                score += value*np.log2(value)
            score=score*(-1)
            #print('Score:',score)
            entropy.append(score)  # Append the Entropy for the given Sample
        #print('Entropy:',entropy)
        self.dataQuality = max(entropy) #Use the max (entropy) value as the data quality index
        #self.dataQuality = sum(entropy)/len(entropy)
        #print("Data Quality for Client: ",self.id," is: ", self.dataQuality)
        #self.dataQuality = (self.dataQuality-3)*100 #<<- All entropys are 3.something so normalized to get the dec values
        #print("Data Quality for Client: ",self.id," is: ", self.dataQuality)
        
    def __setCIF(self):
        #self.clientCIF = int(np.random.randint(0,50,1))  # Random Number between 0 and 49 for check purpose only
        self.__setRoundSINR()
        self.__setPropFair() #This is currently unused
        self.__setdataQuality()
        #self.clientCIF = self.dataQuality + self.roundSINR 
        self.clientCIF = (-1/self.roundSINR) + self.dataQuality #Formula for computing client CIF
        #self.clientCIF = 20
        #print("CIF for Client: ",self.id," is: ", self.clientCIF)
        
    def __getTrainData(self):
        return self.__clientDataX, self.__clientDataY  # Just for Testing will be expanded to select meaningful data
    
    #Public Methods
    def sendActiveStatus(self):
        self.__setActiveStatus()
        return self.activeStatus
    
    def sendCIF(self):
        self.__setCIF()
        return self.clientCIF
    
    def setInitialModel(self,model):
        self.__clientModel=model
        print(self.__clientModel.summary)
         
    def setClientData(self,X,Y):
        self.__clientDataX = X
        self.__clientDataY = Y
   

    def sendClientUpdate(self):
        self.dataCost += self.dataCostFactor
        print("->",self.id,end=" ")
        self.__clientModel.compile(loss = 'sparse_categorical_crossentropy',
                                optimizer = tf.keras.optimizers.SGD (learning_rate=self.lr),
                                metrics=['accuracy'])
        #print(self.__clientDataX.shape)
        #print(self.__clientDataY.shape)
        if len(self.__clientDataX) > 0:
            history = self.__clientModel.fit(self.__clientDataX,
                                   tf.keras.utils.to_categorical(self.__clientDataY),
                                   epochs=self.epochs,
                                   validation_split=0.2,
                                   verbose=0,
                                    shuffle=True)

            updatedWeights = self.__clientModel.get_weights()
        
        else:
            #currentData = next(iter(self.__clientData))  # <-- Shifted to the setDataQuality Function
            history = self.__clientModel.fit(self.__currentData['x'],
                                   self.__currentData['y'],
                                   epochs=self.epochs,
                                   verbose=0,shuffle=True)
            
            
            
            
            updatedWeights = self.__clientModel.get_weights()
       
        
        numOfDataPoints = len(self.__currentData['x'])   #<-- Set the correct type to get the number of datapoints in the current update round

        return (updatedWeights,numOfDataPoints)
    
    #def logTraningHistory(self,history,roundNumber):
    #    loss_history = history.history["loss"]
    #    acc_history = history.history["accuracy"]
    #    numpy_loss_history = numpy.array(loss_history)
    
    def setModelUpdateWeights(self,modelUpdateWeights):
        self.dataCost += self.dataCostFactor
        self.__clientModel.set_weights(modelUpdateWeights)
        #self.__clientModel.
        
    def plotClientData(self):
        #print("Y_Data",self.__clientDataY)
        id0=[]
        id1=[]
        id2=[]
        id3=[]
        id4=[]
        id5=[]
        id6=[]
        id7=[]
        id8=[]
        id9=[]
        for i in range (0,len(self.__clientData['x'])):
            for j in range (0,10):
                if j==self.__clientData['y']:
                    eval('id'+str(j)).append(i)
        xAxis=['0','1','2','3','4','5','6','7','8','9']
        yAxis=[len(id0),len(id1),len(id2),len(id3),len(id4),len(id5),len(id6),len(id7),len(id8),len(id9)]
        plt.bar(xAxis,yAxis)
        title="Data for Client"+str(self.id)
        plt.title(title)
        plt.show()
    
    def getDataCost(self):
        return self.dataCost

### Initialize Clients ###

In [21]:
client_list = {}

# Set the number of clients
num_clients = 3382
for i in range(num_clients):
    name = "client_" + str(i)
    client_list[name] = client(i)
    print("C-id:",i,"->",end=" ")


C-id: 0 -> C-id: 1 -> C-id: 2 -> C-id: 3 -> C-id: 4 -> C-id: 5 -> C-id: 6 -> C-id: 7 -> C-id: 8 -> C-id: 9 -> C-id: 10 -> C-id: 11 -> C-id: 12 -> C-id: 13 -> C-id: 14 -> C-id: 15 -> C-id: 16 -> C-id: 17 -> C-id: 18 -> C-id: 19 -> C-id: 20 -> C-id: 21 -> C-id: 22 -> C-id: 23 -> C-id: 24 -> C-id: 25 -> C-id: 26 -> C-id: 27 -> C-id: 28 -> C-id: 29 -> C-id: 30 -> C-id: 31 -> C-id: 32 -> C-id: 33 -> C-id: 34 -> C-id: 35 -> C-id: 36 -> C-id: 37 -> C-id: 38 -> C-id: 39 -> C-id: 40 -> C-id: 41 -> C-id: 42 -> C-id: 43 -> C-id: 44 -> C-id: 45 -> C-id: 46 -> C-id: 47 -> C-id: 48 -> C-id: 49 -> C-id: 50 -> C-id: 51 -> C-id: 52 -> C-id: 53 -> C-id: 54 -> C-id: 55 -> C-id: 56 -> C-id: 57 -> C-id: 58 -> C-id: 59 -> C-id: 60 -> C-id: 61 -> C-id: 62 -> C-id: 63 -> C-id: 64 -> C-id: 65 -> C-id: 66 -> C-id: 67 -> C-id: 68 -> C-id: 69 -> C-id: 70 -> C-id: 71 -> C-id: 72 -> C-id: 73 -> C-id: 74 -> C-id: 75 -> C-id: 76 -> C-id: 77 -> C-id: 78 -> C-id: 79 -> C-id: 80 -> C-id: 81 -> C-id: 82 -> C-id: 83 -> C-

C-id: 640 -> C-id: 641 -> C-id: 642 -> C-id: 643 -> C-id: 644 -> C-id: 645 -> C-id: 646 -> C-id: 647 -> C-id: 648 -> C-id: 649 -> C-id: 650 -> C-id: 651 -> C-id: 652 -> C-id: 653 -> C-id: 654 -> C-id: 655 -> C-id: 656 -> C-id: 657 -> C-id: 658 -> C-id: 659 -> C-id: 660 -> C-id: 661 -> C-id: 662 -> C-id: 663 -> C-id: 664 -> C-id: 665 -> C-id: 666 -> C-id: 667 -> C-id: 668 -> C-id: 669 -> C-id: 670 -> C-id: 671 -> C-id: 672 -> C-id: 673 -> C-id: 674 -> C-id: 675 -> C-id: 676 -> C-id: 677 -> C-id: 678 -> C-id: 679 -> C-id: 680 -> C-id: 681 -> C-id: 682 -> C-id: 683 -> C-id: 684 -> C-id: 685 -> C-id: 686 -> C-id: 687 -> C-id: 688 -> C-id: 689 -> C-id: 690 -> C-id: 691 -> C-id: 692 -> C-id: 693 -> C-id: 694 -> C-id: 695 -> C-id: 696 -> C-id: 697 -> C-id: 698 -> C-id: 699 -> C-id: 700 -> C-id: 701 -> C-id: 702 -> C-id: 703 -> C-id: 704 -> C-id: 705 -> C-id: 706 -> C-id: 707 -> C-id: 708 -> C-id: 709 -> C-id: 710 -> C-id: 711 -> C-id: 712 -> C-id: 713 -> C-id: 714 -> C-id: 715 -> C-id: 716 ->

C-id: 1256 -> C-id: 1257 -> C-id: 1258 -> C-id: 1259 -> C-id: 1260 -> C-id: 1261 -> C-id: 1262 -> C-id: 1263 -> C-id: 1264 -> C-id: 1265 -> C-id: 1266 -> C-id: 1267 -> C-id: 1268 -> C-id: 1269 -> C-id: 1270 -> C-id: 1271 -> C-id: 1272 -> C-id: 1273 -> C-id: 1274 -> C-id: 1275 -> C-id: 1276 -> C-id: 1277 -> C-id: 1278 -> C-id: 1279 -> C-id: 1280 -> C-id: 1281 -> C-id: 1282 -> C-id: 1283 -> C-id: 1284 -> C-id: 1285 -> C-id: 1286 -> C-id: 1287 -> C-id: 1288 -> C-id: 1289 -> C-id: 1290 -> C-id: 1291 -> C-id: 1292 -> C-id: 1293 -> C-id: 1294 -> C-id: 1295 -> C-id: 1296 -> C-id: 1297 -> C-id: 1298 -> C-id: 1299 -> C-id: 1300 -> C-id: 1301 -> C-id: 1302 -> C-id: 1303 -> C-id: 1304 -> C-id: 1305 -> C-id: 1306 -> C-id: 1307 -> C-id: 1308 -> C-id: 1309 -> C-id: 1310 -> C-id: 1311 -> C-id: 1312 -> C-id: 1313 -> C-id: 1314 -> C-id: 1315 -> C-id: 1316 -> C-id: 1317 -> C-id: 1318 -> C-id: 1319 -> C-id: 1320 -> C-id: 1321 -> C-id: 1322 -> C-id: 1323 -> C-id: 1324 -> C-id: 1325 -> C-id: 1326 -> C-id: 

C-id: 1843 -> C-id: 1844 -> C-id: 1845 -> C-id: 1846 -> C-id: 1847 -> C-id: 1848 -> C-id: 1849 -> C-id: 1850 -> C-id: 1851 -> C-id: 1852 -> C-id: 1853 -> C-id: 1854 -> C-id: 1855 -> C-id: 1856 -> C-id: 1857 -> C-id: 1858 -> C-id: 1859 -> C-id: 1860 -> C-id: 1861 -> C-id: 1862 -> C-id: 1863 -> C-id: 1864 -> C-id: 1865 -> C-id: 1866 -> C-id: 1867 -> C-id: 1868 -> C-id: 1869 -> C-id: 1870 -> C-id: 1871 -> C-id: 1872 -> C-id: 1873 -> C-id: 1874 -> C-id: 1875 -> C-id: 1876 -> C-id: 1877 -> C-id: 1878 -> C-id: 1879 -> C-id: 1880 -> C-id: 1881 -> C-id: 1882 -> C-id: 1883 -> C-id: 1884 -> C-id: 1885 -> C-id: 1886 -> C-id: 1887 -> C-id: 1888 -> C-id: 1889 -> C-id: 1890 -> C-id: 1891 -> C-id: 1892 -> C-id: 1893 -> C-id: 1894 -> C-id: 1895 -> C-id: 1896 -> C-id: 1897 -> C-id: 1898 -> C-id: 1899 -> C-id: 1900 -> C-id: 1901 -> C-id: 1902 -> C-id: 1903 -> C-id: 1904 -> C-id: 1905 -> C-id: 1906 -> C-id: 1907 -> C-id: 1908 -> C-id: 1909 -> C-id: 1910 -> C-id: 1911 -> C-id: 1912 -> C-id: 1913 -> C-id: 

C-id: 2431 -> C-id: 2432 -> C-id: 2433 -> C-id: 2434 -> C-id: 2435 -> C-id: 2436 -> C-id: 2437 -> C-id: 2438 -> C-id: 2439 -> C-id: 2440 -> C-id: 2441 -> C-id: 2442 -> C-id: 2443 -> C-id: 2444 -> C-id: 2445 -> C-id: 2446 -> C-id: 2447 -> C-id: 2448 -> C-id: 2449 -> C-id: 2450 -> C-id: 2451 -> C-id: 2452 -> C-id: 2453 -> C-id: 2454 -> C-id: 2455 -> C-id: 2456 -> C-id: 2457 -> C-id: 2458 -> C-id: 2459 -> C-id: 2460 -> C-id: 2461 -> C-id: 2462 -> C-id: 2463 -> C-id: 2464 -> C-id: 2465 -> C-id: 2466 -> C-id: 2467 -> C-id: 2468 -> C-id: 2469 -> C-id: 2470 -> C-id: 2471 -> C-id: 2472 -> C-id: 2473 -> C-id: 2474 -> C-id: 2475 -> C-id: 2476 -> C-id: 2477 -> C-id: 2478 -> C-id: 2479 -> C-id: 2480 -> C-id: 2481 -> C-id: 2482 -> C-id: 2483 -> C-id: 2484 -> C-id: 2485 -> C-id: 2486 -> C-id: 2487 -> C-id: 2488 -> C-id: 2489 -> C-id: 2490 -> C-id: 2491 -> C-id: 2492 -> C-id: 2493 -> C-id: 2494 -> C-id: 2495 -> C-id: 2496 -> C-id: 2497 -> C-id: 2498 -> C-id: 2499 -> C-id: 2500 -> C-id: 2501 -> C-id: 

C-id: 3017 -> C-id: 3018 -> C-id: 3019 -> C-id: 3020 -> C-id: 3021 -> C-id: 3022 -> C-id: 3023 -> C-id: 3024 -> C-id: 3025 -> C-id: 3026 -> C-id: 3027 -> C-id: 3028 -> C-id: 3029 -> C-id: 3030 -> C-id: 3031 -> C-id: 3032 -> C-id: 3033 -> C-id: 3034 -> C-id: 3035 -> C-id: 3036 -> C-id: 3037 -> C-id: 3038 -> C-id: 3039 -> C-id: 3040 -> C-id: 3041 -> C-id: 3042 -> C-id: 3043 -> C-id: 3044 -> C-id: 3045 -> C-id: 3046 -> C-id: 3047 -> C-id: 3048 -> C-id: 3049 -> C-id: 3050 -> C-id: 3051 -> C-id: 3052 -> C-id: 3053 -> C-id: 3054 -> C-id: 3055 -> C-id: 3056 -> C-id: 3057 -> C-id: 3058 -> C-id: 3059 -> C-id: 3060 -> C-id: 3061 -> C-id: 3062 -> C-id: 3063 -> C-id: 3064 -> C-id: 3065 -> C-id: 3066 -> C-id: 3067 -> C-id: 3068 -> C-id: 3069 -> C-id: 3070 -> C-id: 3071 -> C-id: 3072 -> C-id: 3073 -> C-id: 3074 -> C-id: 3075 -> C-id: 3076 -> C-id: 3077 -> C-id: 3078 -> C-id: 3079 -> C-id: 3080 -> C-id: 3081 -> C-id: 3082 -> C-id: 3083 -> C-id: 3084 -> C-id: 3085 -> C-id: 3086 -> C-id: 3087 -> C-id: 

### Initialize Server ###

In [22]:
# Initialize server with number of clients
serverFA = server(num_clients)

### Initial Broadcast from Server to Client ###

In [23]:
serverFA.initialBroadcast()

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

...

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x000002B27D86AAC8>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

In [ ]:
trainingName = "CIF_UE_MaxFrac0.9_C3382R50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
keys = ['loss','accuracy']
numberOfTraingRounds = 25
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        serverFA.getClientActiveStatus()
        serverFA.getClientCIF()
        serverFA.getClientSelectedMaxFrac()
        serverFA.getModelUpdateFromClients()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)


--------------------------------------------
 Round NO: 0
RoundMaxCIF: 3.2774440981315123  || RoundLimitCIF: 2.9496996883183613  || numberOfClientSlected: 1699


-> 263 -> 1361 -> 476 -> 3309 -> 1423 -> 2711 -> 2800 -> 3118 -> 2795 -> 823 -> 2451 -> 67 -> 2663 -> 3160 -> 2879 -> 1555 -> 1728 -> 15 -> 3264 -> 1658 -> 2759 -> 1690 -> 3376 -> 3046 -> 2469 -> 1640 -> 2474 -> 2523 -> 1957 -> 3255 -> 2750 -> 1803 -> 227 -> 431 -> 2938 -> 326 -> 1463 -> 2286 -> 2963 -> 433 -> 923 -> 1920 -> 853 -> 29 -> 168 -> 2063 -> 320 -> 884 -> 1919 -> 1380 -> 2060 -> 229 -> 3338 -> 1623 -> 2311 -> 315 -> 2310 -> 1306 -> 2440 -> 1733 -> 2249 -> 1456 -> 2939 -> 2789 -> 845 -> 946 -> 2627 -> 644 -> 3011 -> 1072 -> 388 -> 2464 -> 2703 -> 108 -> 2828 -> 603 -> 947 -> 1372 -> 1783 -> 2465 -> 2876 -> 2946 -> 1993 -> 1400 -> 352 -> 1616 -> 106 -> 814 -> 2107 -> 3330 -> 2505 -> 163 -> 2743 -> 1 -> 1662 -> 2942 -> 3279 -> 466 -> 2176 -> 739 -> 2429 -> 2696 -> 2923 -> 893 -> 897 -> 1263 -> 3042 -> 712 -> 2786 -> 1415 -> 2425 -> 2896 -> 2170 -> 3379 -> 1515 -> 2186 -> 2368 -> 3110 -> 482 -> 1486 -> 2207 -> 556 -> 740 -> 696 -> 989 -> 1337 -> 384 -> 1183 -> 234 -> 849 -> 1038

-> 1471 -> 2127 -> 2173 -> 199 -> 746 -> 1220 -> 1872 -> 2977 -> 902 -> 1547 -> 1237 -> 342 -> 1913 -> 618 -> 855 -> 1479 -> 2470 -> 1281 -> 2152 -> 2416 -> 2033 -> 3025 -> 123 -> 1119 -> 3029 -> 3206 -> 1580 -> 1771 -> 3053 -> 121 -> 1323 -> 1312 -> 2714 -> 2806 -> 2082 -> 405 -> 648 -> 888 -> 1439 -> 2180 -> 2779 -> 25 -> 218 -> 1217 -> 3086 -> 1033 -> 2859 -> 3308 -> 852 -> 1088 -> 3005 -> 451 -> 1558 -> 2159 -> 2484 -> 2947 -> 3147 -> 1696 -> 982 -> 2664 -> 3216 -> 2191 -> 3327 -> 42 -> 311 -> 1594 -> 2406 -> 3033 -> 389 -> 1031 -> 1997 -> 477 -> 576 -> 1063 -> 133 -> 3319 -> 557 -> 1980 -> 861 -> 1021 -> 2287 -> 1755 -> 2609 -> 1939 -> 316 -> 1910 -> 1928 -> 2275 -> 3059 -> 2094 -> 142 -> 193 -> 604 -> 2074 -> 1128 -> 1153 -> 1450 -> 2654 -> 3348 -> 815 -> 965 -> 2919 -> 2976 -> 3161 -> 967 -> 2034 -> 2062 -> 2957 -> 3072 -> 3357 -> 1657 -> 3140 -> 424 -> 1174 -> 1249 -> 970 -> 1764 -> 1714 -> 2272 -> 2761 -> 964 -> 1536 -> 2027 -> 3226 -> 3345 -> 1877 -> 1921 -> 2042 -> 2605 -> 2

In [ ]:
plt.plot(serverTestMetrics)


In [ ]:
serverFA.getDataCost()

In [11]:
%tensorboard --logdir logs/ 

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 9596.

In [ ]:
testAcc=[]*numberOfTraingRounds
testLoss=[]*numberOfTraingRounds
for i in range(0,numberOfTraingRounds):
    testAcc.append(trainingMetrics[i][1])
    testLoss.append(trainingMetrics[i][0])
k = np.array(testLoss)

In [ ]:
%tensorboard --kill 30696

In [ ]:
# This Cell is used to get results for Server Only Training
#I.e. there are no clients and all the client dataset (Both Test and Train) are in server
# The server performs 10 epochs in one round and 50 round updates (self, no clients involved)
# The server train data set uses client IDs 1-100 same as federated learning scenario
# The server test dataset uses client IDs 80-100 same as federated learning scenario


trainingName = "ServerOnly_C1_100E10R50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
#Clients Data used from IDs 1 to 100 as a part of server train and test data
keys = ['loss','accuracy']
numberOfTraingRounds = 50
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        print('ROUNDNO:',i)
        serverFA.performServerTraining()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)

In [9]:
# This cell is used to for pretraining demonstration
# The server model is pretrained for 50 epochs from the training dataset of Client IDs 2000-2500 before initial broadcast
# The server test dataset is kept same as before i.e. 80-500
# Since the dataset is in iterable format, the server pretraining is performed using 50 self round updates with 1 epochs each

#---- Pre-training of Server Model
for i in range(0,50):
    #print('ROUNDNO:',i)
    serverFA.performServerTraining()


# BroadCast the Model ( Do not broadcast the model before)
serverFA.initialBroadcast()

#--- Perform FA training
trainingName = "PreTrain2_UE_MaxC10_C100R50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
keys = ['loss','accuracy']
numberOfTraingRounds = 50
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        serverFA.getClientActiveStatus()
        serverFA.getClientCIF()
        serverFA.getClientSelected()
        serverFA.getModelUpdateFromClients()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)


<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x0000026BE8C8E288>>
<bound method Model.summary of <tensorflow.python.keras.engine.sequentia

CIF for Client:  9  is:  13.071611657440663
CIF for Client:  10  is:  19.307250199615954
CIF for Client:  15  is:  16.655866373479366
CIF for Client:  16  is:  16.547020640075207
CIF for Client:  17  is:  12.1024979531765
CIF for Client:  18  is:  16.471421974897385
CIF for Client:  22  is:  18.796480627059935
CIF for Client:  23  is:  20.505327772200108
CIF for Client:  25  is:  12.480404996573926
CIF for Client:  28  is:  11.699947126209736
CIF for Client:  31  is:  19.109511826634407
CIF for Client:  33  is:  15.387672859430314
CIF for Client:  39  is:  17.221995826363564
CIF for Client:  40  is:  11.772256610691548
CIF for Client:  42  is:  14.922313461005688
CIF for Client:  44  is:  18.53044706314802
CIF for Client:  45  is:  15.970082181096076
CIF for Client:  46  is:  20.44484706580639
CIF for Client:  49  is:  18.894258270561696
CIF for Client:  51  is:  16.489815445840357
CIF for Client:  52  is:  17.74684448361397
CIF for Client:  54  is:  17.251078225970268
CIF for Client: 

CIF for Client:  2  is:  22.471538734436034
CIF for Client:  3  is:  14.464562348127366
CIF for Client:  5  is:  17.322243400812148
CIF for Client:  6  is:  18.25001832008362
CIF for Client:  9  is:  13.541529126167298
CIF for Client:  10  is:  22.412965064048766
CIF for Client:  11  is:  17.930554659366607
CIF for Client:  13  is:  17.940068566799162
CIF for Client:  16  is:  20.650224163532258
CIF for Client:  18  is:  16.87945401191711
CIF for Client:  21  is:  20.617119716405867
CIF for Client:  23  is:  19.12706925988197
CIF for Client:  24  is:  14.997074764966964
CIF for Client:  25  is:  13.183637342453004
CIF for Client:  26  is:  17.94539581477642
CIF for Client:  29  is:  16.109444485902785
CIF for Client:  30  is:  20.764278362989426
CIF for Client:  31  is:  22.016575436592102
CIF for Client:  32  is:  12.321460562944413
CIF for Client:  42  is:  14.960877632498741
CIF for Client:  44  is:  22.503812273740767
CIF for Client:  46  is:  23.003350834846497
CIF for Client:  47

CIF for Client:  16  is:  20.282641446590425
CIF for Client:  17  is:  14.143991907238961
CIF for Client:  18  is:  16.72546685278416
CIF for Client:  19  is:  17.389914268255232
CIF for Client:  20  is:  15.05438774585724
CIF for Client:  21  is:  19.09833334028721
CIF for Client:  22  is:  19.328510023355484
CIF for Client:  24  is:  12.385280593633652
CIF for Client:  25  is:  14.521980960965157
CIF for Client:  28  is:  14.94136364042759
CIF for Client:  32  is:  11.443878825306893
CIF for Client:  34  is:  18.64146885752678
CIF for Client:  36  is:  13.811494262814522
CIF for Client:  39  is:  17.961085862517358
CIF for Client:  42  is:  14.684000746011733
CIF for Client:  43  is:  14.49904183268547
CIF for Client:  44  is:  20.488021464347838
CIF for Client:  46  is:  22.524546487927438
CIF for Client:  48  is:  16.4638953858614
CIF for Client:  49  is:  17.35795205116272
CIF for Client:  50  is:  16.683794511556627
CIF for Client:  51  is:  17.124206770062447
CIF for Client:  52

CIF for Client:  11  is:  18.105156191289424
CIF for Client:  16  is:  19.678611460328103
CIF for Client:  17  is:  12.705185419917107
CIF for Client:  21  is:  20.738763719797134
CIF for Client:  22  is:  20.21574535429478
CIF for Client:  23  is:  18.773183579444883
CIF for Client:  24  is:  14.33208484172821
CIF for Client:  25  is:  13.744861208200454
CIF for Client:  30  is:  20.268170716166495
CIF for Client:  32  is:  13.83837826371193
CIF for Client:  33  is:  16.977455421090127
CIF for Client:  37  is:  16.30027118086815
CIF for Client:  38  is:  15.4268623816967
CIF for Client:  39  is:  19.380104516744613
CIF for Client:  40  is:  16.99285390436649
CIF for Client:  41  is:  19.89364548921585
CIF for Client:  42  is:  14.590197492837905
CIF for Client:  43  is:  14.444742437899112
CIF for Client:  44  is:  21.325060573816298
CIF for Client:  47  is:  13.660228359103202
CIF for Client:  48  is:  12.2996069252491
CIF for Client:  49  is:  18.46577175796032
CIF for Client:  50  

...

CIF for Client:  46  is:  20.768708313778042
CIF for Client:  47  is:  14.458182465992868
CIF for Client:  49  is:  18.54211331576109
CIF for Client:  50  is:  16.144217407107355
CIF for Client:  54  is:  18.53120931982994
CIF for Client:  55  is:  16.01348040530458
CIF for Client:  56  is:  20.900199602097274
CIF for Client:  57  is:  18.14121281120926
CIF for Client:  58  is:  15.038217428177596
CIF for Client:  61  is:  16.669363347887995
CIF for Client:  62  is:  15.742757170200347
CIF for Client:  63  is:  19.004234895408153
CIF for Client:  64  is:  22.18061201393604
CIF for Client:  65  is:  13.442767669558526
CIF for Client:  66  is:  19.500150834619998
CIF for Client:  67  is:  15.334986579418182
CIF for Client:  68  is:  19.952926994264125
CIF for Client:  70  is:  14.604571756273508
CIF for Client:  71  is:  16.61142511129379
CIF for Client:  73  is:  16.55516241632402
CIF for Client:  74  is:  12.653363505601883
CIF for Client:  75  is:  19.014473418593408
CIF for Client:  

CIF for Client:  35  is:  9.993862832784654
CIF for Client:  36  is:  12.060374957919121
CIF for Client:  41  is:  17.29149911224842
CIF for Client:  43  is:  15.531164738312363
CIF for Client:  45  is:  16.849452155828477
CIF for Client:  46  is:  20.306225441992282
CIF for Client:  50  is:  14.536116847991943
CIF for Client:  51  is:  14.636491962969304
CIF for Client:  52  is:  16.01474625170231
CIF for Client:  53  is:  17.694420456439257
CIF for Client:  55  is:  14.265526992175728
CIF for Client:  57  is:  20.628830775395034
CIF for Client:  58  is:  16.82856897622347
CIF for Client:  66  is:  17.9572191002965
CIF for Client:  67  is:  19.648456725478173
CIF for Client:  70  is:  14.87520975485444
CIF for Client:  71  is:  17.588321100473404
CIF for Client:  72  is:  19.172814927510917
CIF for Client:  73  is:  16.510456895530226
CIF for Client:  79  is:  15.04235508352518
CIF for Client:  81  is:  18.272137338519094
CIF for Client:  82  is:  15.897139652594923
CIF for Client:  8

CIF for Client:  24  is:  12.877713231146336
CIF for Client:  39  is:  17.045958002805712
CIF for Client:  43  is:  15.976733306720853
CIF for Client:  45  is:  15.475552004277706
CIF for Client:  46  is:  20.59761059820652
CIF for Client:  47  is:  14.160122565329075
CIF for Client:  48  is:  13.728944271951914
CIF for Client:  50  is:  15.789539803862572
CIF for Client:  51  is:  17.247727686464785
CIF for Client:  52  is:  17.57230974137783
CIF for Client:  57  is:  17.644653315842152
CIF for Client:  60  is:  16.39982167005539
CIF for Client:  63  is:  17.394936313629152
CIF for Client:  64  is:  23.469629531055688
CIF for Client:  65  is:  13.33768841266632
CIF for Client:  66  is:  16.815305646955967
CIF for Client:  67  is:  19.112479756474496
CIF for Client:  68  is:  19.019176255762577
CIF for Client:  69  is:  20.610858644805848
CIF for Client:  71  is:  16.089035353660584
CIF for Client:  77  is:  19.624396335184574
CIF for Client:  81  is:  18.22960363075137
CIF for Client:

CIF for Client:  17  is:  15.034217315763236
CIF for Client:  18  is:  18.551519200205803
CIF for Client:  19  is:  17.688850898444652
CIF for Client:  20  is:  15.492919272184372
CIF for Client:  21  is:  19.57109018832445
CIF for Client:  24  is:  13.53384731799364
CIF for Client:  26  is:  16.07716081563383
CIF for Client:  27  is:  9.100747266203399
CIF for Client:  37  is:  15.437586083859205
CIF for Client:  38  is:  15.820375159829855
CIF for Client:  39  is:  18.512079776227473
CIF for Client:  41  is:  17.521304878890515
CIF for Client:  42  is:  13.154770414195955
CIF for Client:  49  is:  19.7079565474391
CIF for Client:  51  is:  15.644854910671711
CIF for Client:  53  is:  18.021614134162665
CIF for Client:  57  is:  18.468559134565293
CIF for Client:  61  is:  17.83496894054115
CIF for Client:  65  is:  13.407713794112205
CIF for Client:  67  is:  17.265803639143705
CIF for Client:  69  is:  22.555541580058634
CIF for Client:  73  is:  16.19330928295851
CIF for Client:  7

CIF for Client:  9  is:  13.620370470434427
CIF for Client:  15  is:  16.9473664149642
CIF for Client:  20  is:  13.641590382307768
CIF for Client:  22  is:  20.184976714849473
CIF for Client:  24  is:  13.431426895260811
CIF for Client:  28  is:  14.628502842783927
CIF for Client:  29  is:  17.183019745498896
CIF for Client:  31  is:  20.780179355815054
CIF for Client:  32  is:  11.652918665558099
CIF for Client:  33  is:  16.894142899513245
CIF for Client:  34  is:  17.07721519306302
CIF for Client:  35  is:  12.090473015606403
CIF for Client:  36  is:  11.885517863184214
CIF for Client:  38  is:  15.469482214450837
CIF for Client:  43  is:  15.156714989170432
CIF for Client:  44  is:  22.773931373655795
CIF for Client:  46  is:  20.700338558554648
CIF for Client:  48  is:  15.437579908072948
CIF for Client:  49  is:  19.290068581774833
CIF for Client:  50  is:  17.18204339414835
CIF for Client:  51  is:  17.271579118967054
CIF for Client:  52  is:  15.370942093729973
CIF for Client:

In [35]:
# This cell is used for performing federated averaging with completely random clients NO CIF or Client Active functions are used

trainingName = "COM_RAND_MaxC10_C100R50" #Name of the current Training Scheme Used for Tensorboard and Saving Results
keys = ['loss','accuracy']
numberOfTraingRounds = 50
serverTestMetrics=[]*numberOfTraingRounds
logdir = "logs/" + trainingName + "/"
summary_writer = tf.summary.create_file_writer(logdir)
with summary_writer.as_default():
    for i in range(0,numberOfTraingRounds):
        serverFA.getRandomClients()
        serverFA.getModelUpdateFromClients()
        metrics = serverFA.testServerModel()
        d_metrics = zip(keys,metrics)
        serverTestMetrics.append(metrics)
        serverFA.updateAllClients()
        for name , value in d_metrics:
            tf.summary.scalar(name, value, step=i)


Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [44 45 18 55 82 31 59 45 43 20]
32/32 [==============================] - 0s 1ms/step - loss: 2.2358 - accuracy: 0.3260
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [28 25 96 36  5 72 30 92 58 63]
32/32 [==============================] - 0s 2ms/step - loss: 2.1087 - accuracy: 0.2780
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [23 33 32 23 63 22 29 23 52 82]
32/32 [==============================] - 0s 1ms/step - loss: 2.0613 - accuracy: 0.3340
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [16 20 96 65 67 29 19 81 99 32]
32/32 [==============================] - 0s 1ms/step - loss: 1.9185 - accuracy: 0.5090
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [44 65 75 57 92 13 79 68 29 81]
32/32 [==============================] - 0s 1ms/step - loss: 1.8423 - accuracy: 0.5200
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [77  1 87

32/32 [==============================] - 0s 1ms/step - loss: 0.8583 - accuracy: 0.7910
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [23 51  5 34 26 82 86 57 59  7]
32/32 [==============================] - 0s 1ms/step - loss: 0.8770 - accuracy: 0.7740
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [ 8  5 33 24 36 93 78 40 43  8]
32/32 [==============================] - 0s 1ms/step - loss: 0.8643 - accuracy: 0.7920
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [37 24 55 48 84 47 75  9 60 64]
32/32 [==============================] - 0s 1ms/step - loss: 0.8531 - accuracy: 0.7870
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [15 32 12 11 92 69 27 17 70 44]
32/32 [==============================] - 0s 1ms/step - loss: 0.8633 - accuracy: 0.7690
Active Clients:  []
CIF of Active Clients:  []
Selected Clients:  [99 23 97 13 49 23 31 39 78 68]
32/32 [==============================] - 0s 1ms/step - loss: 0.8582 - accur

In [ ]:
np.savetxt("loss_history.txt", np.array(testLoss))

In [ ]:
meanSINR = int(np.random.normal(15, 2.5, 1))
print(meanSINR)

In [ ]:
np.round((np.random.normal(meanSINR, 1, 1)[0]), 2)

In [ ]:
raise SystemExit("Stop right there!")


In [ ]:
# calculate the information for a dice roll
from math import log2
# probability of the event
p = 1.0 / 6.0
# calculate information for event
h = -log2(p)
# print the result
print('p(x)=%.3f, information: %.3f bits' % (p, h))

In [ ]:
h

In [ ]:
probs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [ ]:
from matplotlib import pyplot
info = [-log2(p) for p in probs]
# plot probability vs information
pyplot.plot(probs, info, marker='.')
pyplot.title('Probability vs Information')
pyplot.xlabel('Probability')
pyplot.ylabel('Information')
pyplot.show()